# Data Wrangling

The Goodreads datasets were all relatively clean and did not require a lot of cleaning. This notebook details my data wrangling process. 

In [1]:
# Import libraries
# ----------------

# Pandas
import pandas as pd

# Matplotlib
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Numpy
import numpy as np

## Data Loading

The data from Goodreads is separated into three separate csv files: 

* **Ratings:** Contains nearly 6 million user ratings from 53424 users 
* **To-Read:** Contains nearly 1 million books that users added to their 'to-read' shelf 
* **Books:** Contains all of the meta data for 10,000 books. The metadata includes: title, author, number of ratings, number of each type of rating, and more 

To start, I will load each dataset into a separate dataframe so that I can clean the data before merging it into a single dataframe.

### Load ratings

In [2]:
raw_ratings = "../data/raw/ratings.csv"
ratings = pd.read_csv(raw_ratings)
print('DataFrame shape: {}'.format(ratings.shape))
ratings.head()

DataFrame shape: (5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


### Load to_read

In [3]:
raw_toread = "../data/raw/to_read.csv"
to_read = pd.read_csv(raw_toread)
print('DataFrame shape: {}'.format(to_read.shape))
to_read.head()

DataFrame shape: (912705, 2)


,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


### Load books

In [4]:
raw_books = "../data/raw/books.csv"
books = pd.read_csv(raw_books)
print('DataFrame shape: {}'.format(books.shape))
books.head()

DataFrame shape: (10000, 23)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


## Data Cleaning

The ratings and to_read datasets were both clean with no missing data. This left the books dataset for cleaning and prep:

In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
book_id                      10000 non-null int64
goodreads_book_id            10000 non-null int64
best_book_id                 10000 non-null int64
work_id                      10000 non-null int64
books_count                  10000 non-null int64
isbn                         9300 non-null object
isbn13                       9415 non-null float64
authors                      10000 non-null object
original_publication_year    9979 non-null float64
original_title               9415 non-null object
title                        10000 non-null object
language_code                8916 non-null object
average_rating               10000 non-null float64
ratings_count                10000 non-null int64
work_ratings_count           10000 non-null int64
work_text_reviews_count      10000 non-null int64
ratings_1                    10000 non-null int64
ratings_2                    10000 n

In [6]:
# Drop columns not of interest
books = books.drop('isbn', 1)
books = books.drop('isbn13', 1)
books = books.drop('image_url', 1)
books = books.drop('small_image_url', 1)

# Fill in the missing publication years with the year 1984
## Since this is a datetime and not a float it does not matter what year I give these missing values
books.original_publication_year = books.original_publication_year.fillna(1984)

# Replace missing original_title features with info from title
books.original_title.fillna(books.title, inplace=True)

# Replace missing language codes with 'unknown'
books.language_code = books.language_code.fillna('unknown')

In [7]:
# Create column for a user's average rating
ratings['avg_rating_by_user'] = ratings['rating'].groupby(ratings['user_id']).transform('mean')

# Create a column that counts the number of ratings per user
ratings['number_ratings_per_user'] = ratings['user_id'].groupby(ratings['user_id']).transform('count') 
ratings.head()

,user_id,book_id,rating,avg_rating_by_user,number_ratings_per_user
0,1,258,5,3.589744,117
1,2,4081,4,4.415385,65
2,2,260,5,4.415385,65
3,2,9296,5,4.415385,65
4,2,2318,3,4.415385,65


## Data Merging

Now I am ready to start merging the dataframes. First I will merge the ratings and books dataframes:

In [8]:
read = pd.merge(ratings, books)
print('DataFrame shape: {}'.format(read.shape))
read.head()

DataFrame shape: (5976479, 23)


,user_id,book_id,rating,avg_rating_by_user,number_ratings_per_user,goodreads_book_id,best_book_id,work_id,books_count,authors,...,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,258,5,3.589744,117,1232,1232,3209783,279,"Carlos Ruiz Zafón, Lucia Graves",...,eng,4.24,263685,317554,24652,4789,11769,42214,101612,157170
1,11,258,3,3.591667,120,1232,1232,3209783,279,"Carlos Ruiz Zafón, Lucia Graves",...,eng,4.24,263685,317554,24652,4789,11769,42214,101612,157170
2,143,258,4,3.603774,159,1232,1232,3209783,279,"Carlos Ruiz Zafón, Lucia Graves",...,eng,4.24,263685,317554,24652,4789,11769,42214,101612,157170
3,242,258,5,3.715447,123,1232,1232,3209783,279,"Carlos Ruiz Zafón, Lucia Graves",...,eng,4.24,263685,317554,24652,4789,11769,42214,101612,157170
4,325,258,4,3.689394,132,1232,1232,3209783,279,"Carlos Ruiz Zafón, Lucia Graves",...,eng,4.24,263685,317554,24652,4789,11769,42214,101612,157170


In order to keep track of which books came from the ratings dataset and which came from the to_read dataset, I will create a 'read_unread' column: 

In [9]:
# Create a column that says whether a user has read a particular book 
# Fill all books from the read dataframe as 'read'
read['read_unread'] = 'read'
read.read_unread.value_counts()

read    5976479
Name: read_unread, dtype: int64

Next I will merge the to_read dataframe with the books dataframe:

In [10]:
# Merge the to_read dataframe with the books dataframe
toread = pd.merge(to_read, books)
print('DataFrame shape: {}'.format(toread.shape))

DataFrame shape: (912705, 20)


I will then update the read_unread column to account for the data that came from the to_read dataset:

In [11]:
# Create a read_unread column to note which rows came from the to_read dataframe
toread['read_unread'] = np.nan

# Concatenate read and to_read dataframes
df = read.append(toread, sort=True)
print('DataFrame shape: {}'.format(df.shape))

DataFrame shape: (6889184, 24)


In [12]:
df.read_unread = df['read_unread'].fillna('to read')
df.read_unread.value_counts()

read       5976479
to read     912705
Name: read_unread, dtype: int64

I need to reduce the amount of data I will be working with since I do not want to work with a dataframe with over 6 million rows. 

I will be reducing the dataframe to only include the users that have reviewed at least 145 books and books that have at least 100,000 reviews.

In [13]:
# Filter the dataset to users with more than 145 ratings  
df = df[df['number_ratings_per_user'] > 145]
df = df.drop('number_ratings_per_user', 1)
print('Number of reviewers in dataset:', len(df.user_id.value_counts()))

# Filter the dataframe for books that have at least 100,000 ratings
df = df[df['work_ratings_count'] > 100000 ]
print('Number of books in dataframe:', len(df.title.value_counts()))

Number of reviewers in dataset: 5423
Number of books in dataframe: 1049


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504756 entries, 2 to 5968046
Data columns (total 23 columns):
authors                      504756 non-null object
average_rating               504756 non-null float64
avg_rating_by_user           504756 non-null float64
best_book_id                 504756 non-null int64
book_id                      504756 non-null int64
books_count                  504756 non-null int64
goodreads_book_id            504756 non-null int64
language_code                504756 non-null object
original_publication_year    504756 non-null float64
original_title               504756 non-null object
rating                       504756 non-null float64
ratings_1                    504756 non-null int64
ratings_2                    504756 non-null int64
ratings_3                    504756 non-null int64
ratings_4                    504756 non-null int64
ratings_5                    504756 non-null int64
ratings_count                504756 non-null int64
read_unrea

## Data Exporting

In [16]:
# Save the cleaned up dataset to a csv
processed_df = '../data/processed/goodreads.csv'
df.to_csv(processed_df)